In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
hf_token = os.environ.get("hf_token")

from huggingface_hub import login
if hf_token :
    print("available hf token")
    login(token=hf_token) 
else :
    print("No hf token")

# 1. 설정 (사용 중인 경로 및 ID)
base_model_id = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
adapter_repo_id = "JungSeong2/library-QA-Adapter"
revision = "v1.0"  # 특정 버전(브랜치) 지정
save_path = "./LoRA" # 저장할 로컬 디렉토리

# 폴더가 없으면 생성
os.makedirs(save_path, exist_ok=True)

print(f"--- 로딩 시작: {adapter_repo_id} (revision: {revision}) ---")

# 2. 토크나이저 로드 및 저장
# 어댑터 저장소에 올린 토크나이저를 가져와서 지정된 경로에 저장합니다.
tokenizer = AutoTokenizer.from_pretrained(adapter_repo_id, revision=revision, token=hf_token)
tokenizer.save_pretrained(save_path)
print(f"[1/3] 토크나이저 저장 완료: {save_path}")

# 3. 베이스 모델 로드
# 저장만 하는 목적이라면 양자화 없이 로드하는 것이 나중에 활용도가 높습니다.
# 메모리가 부족하다면 device_map="cpu"를 고려하세요.
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
print(f"[2/3] 베이스 모델 로드 완료")

# 4. 어댑터(LoRA) 로드 및 저장
# 베이스 모델 위에 어댑터를 올린 뒤, save_pretrained를 호출합니다.
model = PeftModel.from_pretrained(base_model, adapter_repo_id, revision=revision)
model.save_pretrained(save_path)
print(f"[3/3] 어댑터(LoRA) 저장 완료: {save_path}")

print("-" * 50)
print(f"작업 완료! 모든 파일이 '{os.path.abspath(save_path)}'에 저장되었습니다.")

available hf token
--- 로딩 시작: JungSeong2/library-QA-Adapter (revision: v1.0) ---
[1/3] 토크나이저 저장 완료: ./LoRA


`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

/home/vsc/LLM_TUNE/QA-FineTune/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:435: UserWarning: 
    Found GPU0 NVIDIA GB10 which is of cuda capability 12.1.
    Minimum and Maximum cuda capability supported by this version of PyTorch is
    (8.0) - (12.0)
    
  queued_call()


Loading weights:   0%|          | 0/272 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

[2/3] 베이스 모델 로드 완료


adapter_config.json:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

NotImplementedError: `get_input_embeddings` not auto‑handled for ExaoneModel; please override in the subclass.